# **Clipboard Challenge**

Metrics can be classified into Healthcare facilities (“HCFs”) metrics and healthcare professionals (“HCPs”), lets consider one by one 

In [6]:
import polars as pl

df = pl.read_csv('pricingAssociateDatasetv2.csv')

## **HCF Metrics**

### *Worker Rating*


In [14]:
status_to_rating = {
    "WORKER_CANCEL": 1,
    "SHIFT_UNBOOK": 1,
    "SHIFT_VERIFY": 4,
    "NO_CALL_NO_SHOW": 0,
}


data = ( 
            df.filter(pl.col('ACTION').is_in(['WORKER_CANCEL','SHIFT_UNBOOK','SHIFT_VERIFY','NO_CALL_NO_SHOW']))
            .select('WORKER_ID','ACTION')
        )

data = (
        data.with_columns([data["ACTION"].apply(lambda x : status_to_rating.get(x)).alias('score')])
            .groupby(pl.col('WORKER_ID')).agg(pl.mean('score').alias('rating'))
)

In [15]:
import plotly.graph_objs as go

worker_data = data.to_pandas()

 
fig = go.Figure()
fig.add_trace(go.Scatter(x=worker_data['WORKER_ID'], y=worker_data['rating'], mode='markers', 
                         marker=dict(color=worker_data['rating'], colorscale='RdYlGn',
        showscale=True)))
fig.update_layout(title='Worker Ratings', xaxis_title='Worker id', yaxis_title='Rating', showlegend=False)
fig.show()

### *Shift action rate*
    Following are calculated.
    - Shift fill rate
    - Worker cancel rate
    - worker violation rate
    - facility cancelation rate

In [2]:
fill_rate = (
     df.groupby('FACILITY_ID','SHIFT_TYPE').agg(pl.count().alias('total_count'))
     .join(
         df.filter(pl.col('ACTION')=='SHIFT_CLAIM').groupby('FACILITY_ID','SHIFT_TYPE').agg(pl.count().alias('claimed_count')),
         on=['FACILITY_ID','SHIFT_TYPE'],how='left')
     .join(
         df.filter(pl.col('ACTION')=='WORKER_CANCEL').groupby('FACILITY_ID','SHIFT_TYPE').agg(pl.count().alias('worker_cancel_count')),
          on=['FACILITY_ID','SHIFT_TYPE'],how='left')
    .join(
         df.filter(pl.col('ACTION')=='SHIFT_UNBOOK').groupby('FACILITY_ID','SHIFT_TYPE').agg(pl.count().alias('worker_violation_count')),
          on=['FACILITY_ID','SHIFT_TYPE'],how='left')
     .join(
         df.filter((pl.col('ACTION')=='FACILITY_CANCEL')|(pl.col('ACTION')=='SHIFT_DELETE')).groupby('FACILITY_ID','SHIFT_TYPE').agg(pl.count().alias('facility_cancelation_count')),
          on=['FACILITY_ID','SHIFT_TYPE'],how='left')
     .with_columns([
         ((pl.col('claimed_count')/pl.col('total_count'))*100).alias('fill_rate'),
         ((pl.col('worker_cancel_count')/pl.col('total_count'))*100).alias('worker_cancel_rate'),
         ((pl.col('worker_violation_count')/pl.col('total_count'))*100).alias('worker_violation_rate'),
         ((pl.col('facility_cancelation_count')/pl.col('total_count'))*100).alias('facility_cancelation_rate') 
     ]).select('FACILITY_ID','SHIFT_TYPE','fill_rate','worker_cancel_rate','worker_violation_rate','facility_cancelation_rate').sort('FACILITY_ID')
 )

In [5]:
import plotly.express as px

fig = px.area(fill_rate.to_pandas(), 
              x='FACILITY_ID', 
              y=['worker_violation_rate','facility_cancelation_rate','worker_cancel_rate','fill_rate'],
              title="Shift Rates by Facility ID", 
              labels={'value': 'Percentage', 'variable': 'Metric Type', 'FACILITY_ID': 'Facility ID'},
              hover_data={'value': ':.2f'},
              line_group='variable',
              color_discrete_sequence=px.colors.qualitative.Set1,)

fig.show(width=2000)

### *Time taken to fill shifts*
    The following lead times  are calculated
    - shift claim
    - worker cancel  
    - worker violation  
    - facility cancelation

In [8]:
time_rate = (
     df.groupby('FACILITY_ID','SHIFT_TYPE').agg(pl.count().alias('total_count'))
     .join(
         df.filter(pl.col('ACTION')=='SHIFT_CLAIM').groupby('FACILITY_ID','SHIFT_TYPE').agg(pl.mean('LEAD_TIME').alias('avg_claim_time')),
         on=['FACILITY_ID','SHIFT_TYPE'],how='left')
     .join(
         df.filter(pl.col('ACTION')=='WORKER_CANCEL').groupby('FACILITY_ID','SHIFT_TYPE').agg(pl.mean('LEAD_TIME').alias('avg_worker_cancel_time')),
          on=['FACILITY_ID','SHIFT_TYPE'],how='left')
    .join(
         df.filter(pl.col('ACTION')=='SHIFT_UNBOOK').groupby('FACILITY_ID','SHIFT_TYPE').agg(pl.mean('LEAD_TIME').alias('avg_wrkr_voilation_time')),
          on=['FACILITY_ID','SHIFT_TYPE'],how='left')
     .join(
         df.filter((pl.col('ACTION')=='FACILITY_CANCEL')|(pl.col('ACTION')=='SHIFT_DELETE')).groupby('FACILITY_ID','SHIFT_TYPE').agg(pl.mean('LEAD_TIME').alias('avg_facility_cancelation_time')),
          on=['FACILITY_ID','SHIFT_TYPE'],how='left')
     .select('FACILITY_ID','SHIFT_TYPE','avg_claim_time','avg_worker_cancel_time','avg_wrkr_voilation_time','avg_facility_cancelation_time').sort('FACILITY_ID')
 )

In [10]:
fig = px.area(time_rate.to_pandas(), 
              x='FACILITY_ID', 
              y=['avg_wrkr_voilation_time','avg_facility_cancelation_time','avg_claim_time','avg_worker_cancel_time'],
              title="  Time Metrics by Facility ID", 
              labels={'value': 'Time', 'variable': 'Metric Type', 'FACILITY_ID': 'Facility ID'},
              hover_data={'value': ':.2f'},
              line_group='variable',
              color_discrete_sequence=px.colors.qualitative.Set1,)

fig.show(width=2000)

We can use the above metrics to show the shift rates and avg lead times for above actions grouped by shift type

In [12]:
import plotly.graph_objs as go
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

output = widgets.Output()



fdf_fill = fill_rate.to_pandas()
fdf_time = time_rate.to_pandas()

facility_dropdown = widgets.Dropdown(options=fdf_fill['FACILITY_ID'].unique(), description='Facility ID:')


def update_chart(facility_id):
    filtered_df_fill = fdf_fill[fdf_fill['FACILITY_ID'] == facility_id]
    filtered_df_time = fdf_time[fdf_time['FACILITY_ID'] == facility_id]

    fig = make_subplots(rows=1, cols=2, subplot_titles=('Fill Rates', 'Time Rates'))
    
    fig.add_trace(go.Bar(name='Fill Rate', x=filtered_df_fill['SHIFT_TYPE'], y=filtered_df_fill['fill_rate'], offset=0, width=0.15), row=1, col=1)
    fig.add_trace(go.Bar(name='Worker Cancellation Rate', x=filtered_df_fill['SHIFT_TYPE'], y=filtered_df_fill['worker_cancel_rate'], offset=0.2, width=0.15), row=1, col=1)
    fig.add_trace(go.Bar(name='Worker Violation Rate', x=filtered_df_fill['SHIFT_TYPE'], y=filtered_df_fill['worker_violation_rate'], offset=0.4, width=0.15), row=1, col=1)
    fig.add_trace(go.Bar(name='Facility Cancellation Rate', x=filtered_df_fill['SHIFT_TYPE'], y=filtered_df_fill['facility_cancelation_rate'], offset=0.6, width=0.15), row=1, col=1)
    fig.update_xaxes(title_text='Shift Type', row=1, col=1)
    fig.update_yaxes(title_text='Rate', row=1, col=1)
    
    fig.add_trace(go.Bar(name='Avg Claim Time', x=filtered_df_time['SHIFT_TYPE'], y=filtered_df_time['avg_claim_time'], offset=0, width=0.15), row=1, col=2)
    fig.add_trace(go.Bar(name='Avg Worker Cancel Time', x=filtered_df_time['SHIFT_TYPE'], y=filtered_df_time['avg_worker_cancel_time'], offset=0.2, width=0.15), row=1, col=2)
    fig.add_trace(go.Bar(name='Avg Worker Violation Time', x=filtered_df_time['SHIFT_TYPE'], y=filtered_df_time['avg_wrkr_voilation_time'], offset=0.4, width=0.15), row=1, col=2)
    fig.add_trace(go.Bar(name='Avg Facility Cancellation Time', x=filtered_df_time['SHIFT_TYPE'], y=filtered_df_time['avg_facility_cancelation_time'], offset=0.6, width=0.15), row=1, col=2)
    fig.update_xaxes(title_text='Shift Type', row=1, col=2)
    fig.update_yaxes(title_text='Time', row=1, col=2)

    fig.update_layout(
    title='Fill Rates and Times Rates by Shift Type for facility {}'.format(facility_id), 
    xaxis_title='Shift Type', 
    height=600)
    
    with output:
        output.clear_output(wait=True)
        iplot(fig)



def on_facility_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        update_chart(int(change['new']))


display(facility_dropdown)
display(output)
update_chart(fdf_fill['FACILITY_ID'].iloc[33])


facility_dropdown.observe(on_facility_change)

Dropdown(description='Facility ID:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18…

Output()

### *Worker performance*

Calculating the workers with most violations, cancellation and no shows can help to identify them and place a temporary ban or limiting their ability to apply for shifts more often

In [16]:
wk=(
    df.groupby('WORKER_ID').
    apply(lambda data : data.filter(pl.col('ACTION').is_in(['WORKER_CANCEL',
                                                            'SHIFT_UNBOOK','NO_CALL_NO_SHOW'])))
    .groupby('WORKER_ID','ACTION').agg(pl.count()).sort('count',descending=True)
)

In [18]:
dc = wk.to_pandas()
fig = px.sunburst(dc, path=['ACTION', 'WORKER_ID'], values='count', title='Workers with more violations')
fig.show()

----------------------------------

## **HCP Metrics**

### *facility ratings*

Similar to worker ratings , facilities also can be rated based number of shifts posted , shifts filled , cancellation rate , lead time before each facility taken action
Higher shifts posting and filling rates shows the availability of shifts to HCP's , cancellations can be disruptive and costly for HCFs, and they need to be able to rely on the HCPs they've booked. Higer lead time can help HCF's to apply for another shift

In [19]:
import pandas as pd
import numpy as np

grouped_data = df.to_pandas().groupby('FACILITY_ID').agg(
    num_shifts_posted=('SHIFT_ID', 'nunique'),
    num_shifts_filled=('WORKER_ID', 'nunique'),
    avg_lead_time=('LEAD_TIME', 'mean'),
    cancellation_rate=('ACTION', lambda x: (x == 'FACILITY_CANCEL').sum() / len(x))
)

 
normalized_data = (grouped_data - grouped_data.min()) / (grouped_data.max() - grouped_data.min())

 
weights = pd.Series({
    'num_shifts_posted': 0.3,
    'num_shifts_filled': 0.4,
    'avg_lead_time': 0.2,
    'cancellation_rate': 0.1
})

 
rating_scores = (normalized_data * weights).sum(axis=1)

 
num_facilities = len(rating_scores)
top_10_percent = int(num_facilities * 0.1)
top_20_percent = int(num_facilities * 0.2)

rating_scores_ranked = rating_scores.rank(ascending=False)
facilities_ratings = pd.cut(
    rating_scores_ranked,
    bins=[0, top_10_percent, top_20_percent, num_facilities],
    labels=[5, 4, 3]
)

 
grouped_data['rating'] = facilities_ratings
final_data = grouped_data.reset_index()[['FACILITY_ID','rating']]

In [23]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=final_data['FACILITY_ID'], y=final_data['rating'], mode='markers', 
                         marker=dict(color=final_data['rating'], colorscale='RdYlGn',
        showscale=True)))
fig.update_layout(title='Facility Ratings', xaxis_title='Worker id', yaxis_title='Rating', showlegend=False)
fig.show()

In [24]:
lead_time = (
    df.filter(pl.col('ACTION')=='SHIFT_CLAIM')
    .groupby(pl.col('FACILITY_ID')).agg( pl.mean('LEAD_TIME').alias('lead_open_time'))
    .join(
    df.filter(pl.col('ACTION')=='FACILITY_CANCEL_FILLED')
    .groupby(pl.col('FACILITY_ID')).agg( pl.mean('LEAD_TIME').alias('lead_facilty_cancel_time')),on='FACILITY_ID'
    )
)

In [25]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

dx = lead_time.to_pandas()

dx['lead_open_time_days'] = dx['lead_open_time'] / 24
dx['lead_facility_cancel_time_days'] = dx['lead_facilty_cancel_time'] / 24

 
fig = make_subplots(rows=1, cols=2, subplot_titles=("Lead Open Time", "Lead Facility Cancel Time"))

 
fig.add_trace(
    go.Bar(y=dx['FACILITY_ID'], x=dx['lead_open_time_days'], orientation='h'),
    row=1, col=1
)

 
fig.add_trace(
    go.Bar(y=dx['FACILITY_ID'], x=dx['lead_facility_cancel_time_days'], orientation='h'),
    row=1, col=2
)

 
fig.update_layout(
    title_text="Shift open against Shift cancellations(facility) times by facility",
    height=500, width=1000,
    yaxis=dict(title="Facility ID"),
    xaxis=dict(title="Days"),
    showlegend=False
)
 
fig.show()
